## Responses
- Agentic API primitive


In [1]:
import os
OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
OPEN_AI_MODEL = "gpt-4o-mini"

In [2]:
from openai import  OpenAI
import requests, json


client = OpenAI(api_key=OPEN_AI_KEY)

In [3]:
response = client.responses.create(
    model=OPEN_AI_MODEL,
    input=[{
        "role": "user",
        "content": "Write a one-sentence bedtime story about a unicorn."
    }]
)
print(response)

Response(id='resp_681b4ad452cc819184c636a88ae64b9806793b882af0054d', created_at=1746619092.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_681b4ad61d0081919ea63f2367dfdaa906793b882af0054d', content=[ResponseOutputText(annotations=[], text='As the moonlight shimmered on the meadow, a gentle unicorn named Lila spread her wings and soared into the starry sky, spreading dreams of magic and wonder to every child sleeping below.', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=18, input

In [4]:
response.dict()

/tmp/ipykernel_7910/3202056457.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response.dict()


{'id': 'resp_681b4ad452cc819184c636a88ae64b9806793b882af0054d',
 'created_at': 1746619092.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'id': 'msg_681b4ad61d0081919ea63f2367dfdaa906793b882af0054d',
   'content': [{'annotations': [],
     'text': 'As the moonlight shimmered on the meadow, a gentle unicorn named Lila spread her wings and soared into the starry sky, spreading dreams of magic and wonder to every child sleeping below.',
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'max_output_tokens': None,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None, 'summary': None},
 'service_tier': 'default',
 'status': 'completed',
 'text': {'format': {'type': 'text'}},
 'truncation': 'd

### Tool call

In [5]:
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(data)
    return data['current']['temperature_2m']

# print(get_weather(latitude="9.9312", longitude="76.2673"))

In [6]:

tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celcius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

user_query = [{"role": "user", "content": "What is the weather like in Kochi today?"}]
weather_response = client.responses.create(
    model=OPEN_AI_MODEL,
    input=user_query,
    tools=tools,
)

In [7]:
weather_response.output

[ResponseFunctionToolCall(arguments='{"latitude":9.9312,"longitude":76.2673}', call_id='call_cX10XbZX3V6Nhk0pzHBjRWKY', name='get_weather', type='function_call', id='fc_681b4ae3f7888191884ac473b4319fea0c214cec41f68beb', status='completed')]

In [8]:
tool_call = weather_response.output[0]
args = json.loads(tool_call.arguments)

In [9]:
result = globals()[tool_call.name](**args)
result


{'latitude': 9.875, 'longitude': 76.25, 'generationtime_ms': 0.438690185546875, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 6.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'wind_speed_10m': 'km/h'}, 'current': {'time': '2025-05-07T11:45', 'interval': 900, 'temperature_2m': 31.4, 'wind_speed_10m': 14.8}, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'wind_speed_10m': 'km/h'}, 'hourly': {'time': ['2025-05-07T00:00', '2025-05-07T01:00', '2025-05-07T02:00', '2025-05-07T03:00', '2025-05-07T04:00', '2025-05-07T05:00', '2025-05-07T06:00', '2025-05-07T07:00', '2025-05-07T08:00', '2025-05-07T09:00', '2025-05-07T10:00', '2025-05-07T11:00', '2025-05-07T12:00', '2025-05-07T13:00', '2025-05-07T14:00', '2025-05-07T15:00', '2025-05-07T16:00', '2025-05-07T17:00', '2025-05-07T18:00', '2025-05-07T19:00', '2025-05-07T20:00', '2025-05-07T21:00', '2025-05-07T22:00', '2025-05-07T2

31.4